# Prática 06 - Bibliotecas de visualização geográficas

In [20]:
import altair as alt
from vega_datasets import data
import geopandas as gpd
import json

#fp = "data/municipios_simp.json"
fp = "data/estados.json"

A função definida a seguir foi baseada [neste exemplo](https://www.districtdatalabs.com/altair-choropleth-viz). O objetivo é criar um objeto `mapa` com as layers: `base`,`choropleth` e `labels`

In [17]:
# check https://www.districtdatalabs.com/altair-choropleth-viz

def gen_map(geodata, color_column, title, tooltip):
    # Add Base Layer
    selection = alt.selection_multi(fields=[color_column]) 
    base = alt.Chart(geodata, title = title).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
    ).properties(
        width=800,
        height=800
    )
    # Add Choropleth Layer
    choro = alt.Chart(geodata).mark_geoshape(
        fill='lightgray',
        stroke='black'
    ).encode(
        alt.Color(color_column, 
                  type='quantitative', 
                  scale=alt.Scale(scheme='bluegreen'),
                  title = title),
        tooltip=tooltip).add_selection(
        selection
    )
    # Add Labels Layer
    labels = alt.Chart(geodata).mark_text(baseline='top'
     ).properties(
        width=400,
        height=400
     ).encode(
         longitude='properties.centroid_lon:Q',
         latitude='properties.centroid_lat:Q',
         text='properties.SIGLA:O',
         size=alt.value(8),
         opacity=alt.value(1)
     )

    return base + choro + labels

Aqui fazemos a conversão do objeto `GeoJson` em um [Pandas](www.pandas.pydata.org) `DataFrame`

In [21]:
customGeoJson = gpd.read_file(fp)
gdf = gpd.GeoDataFrame.from_features(customGeoJson)

In [12]:
gdf = gdf[gdf['GEOCODIGO'].astype(int)>=20000]

In [22]:
gdf['centroid_lon'] = gdf['geometry'].centroid.x
gdf['centroid_lat'] = gdf['geometry'].centroid.y
gdf.head()

,geometry,GEOCODIGO,GEOMETRIAA,NOME,NOMEABREV,SIGLA,centroid_lon,centroid_lat
0,POLYGON ((-40.52155087297817 -7.31933745906446...,23,Não,Ceará,Ceará,CE,-39.616319,-5.089406
1,(POLYGON ((-29.34587513098174 0.91730372350471...,53,Não,Distrito Federal,Distrito Federal,DF,-47.790753,-15.780375
2,POLYGON ((-40.96056096348602 -21.2974821650153...,32,Não,Espírito Santo,Espírito Santo,ES,-40.671995,-19.580448
3,(POLYGON ((-50.41083280930408 1.86076035569839...,16,Não,Amapá,Amapá,AP,-51.964711,1.441056
4,POLYGON ((-63.37845684336658 2.229662499994111...,14,Não,Roraima,Roraima,RR,-61.388990,2.090681


In [23]:
choro_json = json.loads(gdf.to_json())
choro_data = alt.Data(values=choro_json['features'])

In [24]:
pop_2000_map = gen_map(geodata=choro_data, color_column='GEOCODIGO', title='2000',tooltip=['properties.NOME:O','properties.SIGLA:Q'])
pop_2000_map
html = pop_2000_map.to_html()
with open('predominant_styles_by_neighbhourhood.html', 'w') as f:
    f.write(html)